Recurrent Neural Network is a generalization of a feed-forward neural network with internal memory. Well, ‘recurrent’ sounds complicated and they indeed are. They exhibit temporal dynamic behavior with a sequence dependence. That means RNNs have got to do something with regular or irregular time-based information retrieval over a period of time where the current information is influenced by the previous one in the memory. Most widely known RNNs are Long Short Term Memory (LSTMs). Please Refer LSTMs for detailed info. 

While trying to predict the Stock market trends, one must understand that future stock price is not dependent on past stock prices. Yet, by using the RNNs we can extract the time-based patterns (irregular or regular) for a certain period to evaluate the future expectation of stock being bullish or bearish. There could be various ways of prediction based on the features we choose. For example, analysis of GoogleTrend search volume for 98 terms of varying financial relevance, published in Scientific Reports, suggests that increases in search volume for financially relevant search terms tend to precede large losses in financial markets. For our understanding, we will only focus on time-series learning using LSTMs.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun 18 06:56:31 2020

@author: G.Varchaleswari
"""

import numpy as np
import matplotlib.pyplot as pt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import pandas as pd

#%%import the data
stock_data = pd.read_csv('../input/google-stock-price/Google_Stock_Price_Train.csv')
data_train = stock_data.iloc[:,1:2]
data_train = np.array(data_train)


In [ ]:

#%%feature scaling
msc = MinMaxScaler()
data_train = msc.fit_transform(data_train)

Data preparation for the LSTM network includes time steps. Some sequence problems may have a varied number of time steps per sample. You may have certain features of a dataset determining its relevant outcome. For example, a player's victory or loss in a game is contingent on the number of moves and the type of moves he does in the game, Each move would be a sample of the observations that lead up to the event and that would be the time steps, and the variables observed would be the features (types and number of moves). Instead of feeding the past observations as separate input features, we can use them as time steps of one input feature. This helps the RNN to build its internal memory.

The choice of the number of timesteps to implement depends on the learning rate and loss values of the algorithm. We want the loss to be minimized but not as much as to 0.0 percent as this may lead to overfitting. By experimenting with the number of time steps, 60 seemed most effective in my case. 

The code below runs a for loop with the range of input data with 60 intervals for training each value where one time step is one point of observation in the sample. Every 60 values before the given values become the input x  while the 61st value becomes label y. This process is repeated until we reach the last value in the dataset.


In [ ]:
#%% Creating a data structure with 60 timestaps and 1 output
x_train = []
y_train = []

for i in range(60, 1258):
    x_train.append(data_train[i-60:i, 0])
    y_train.append(data_train[i,0])
    

x_train = np.array(x_train)
y_train = np.array(y_train)    


The three dimensions of this input are:
Samples - One sequence is one sample. A batch is comprised of one or more samples. Time Steps - One time step is one point of observation in the sample. Features - One feature is one observation at a time step. The LSTM input layer in keras.layers.LSTM expects a 3D array of data when fitting the model and when making predictions, even if specific dimensions of the array contain a single value, e.g. one sample or one feature.

After executing the below code our x_train.shape should return (1198, 60, 1) where 1198 are the number of samples, 60 is the number of time steps and 1 is the number of features.

In [ ]:
#%% reshaping
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


We will build a sequential model for LSTM. Input_shape will be (60,1). The input_shape will need us to enter the number of timesteps and the features as the size. The size of the samples will be taken while fitting the model.

In [ ]:
#%% building the rnn

model = tf.keras.models.Sequential()


LSTM (https://keras.io/api/layers/recurrent_layers/lstm/) layer is defined by using keras.layers. LSTM constructor will be having two most important arguments while adding the first layer:
units: Positive integer, the dimensionality of the output space.
return_sequences: Boolean. Whether to return the last output. in the output sequence, or the full sequence. Default: False.
Dropout: Applies Dropout to the input. Rate - Float between 0 and 1. The fraction of the units to drop for the linear transformation of the inputs. Default: 0.


In [ ]:
#%% Adding LSTM layer and the regularization dropout
model.add(tf.keras.layers.LSTM(units =  70, return_sequences = True, input_shape = (x_train.shape[1],1)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.LSTM(units =  70, return_sequences = True))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.LSTM(units =  70, return_sequences = True))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.LSTM(units =  70))
model.add(tf.keras.layers.Dropout(0.25))


In [ ]:
#%% adding the output layer
model.add(tf.keras.layers.Dense(units = 1))


In [ ]:
#%% compile the model
model.compile(optimizer = 'adam', loss='mean_squared_error')


In [ ]:

#%% fit the model
model.fit(x_train, y_train, epochs = 100, batch_size = 32 )


In [ ]:
#%% prediction
dataset_test = pd.read_csv('../input/google-stock-price/Google_Stock_Price_Test.csv')
dataset_test = dataset_test.iloc[:, 1:2].values

dataset_total = pd.concat([pd.DataFrame(data_train), pd.DataFrame(dataset_test)], axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = msc.transform(inputs)
x_test = []
for i in range(60, 80):
    x_test.append(inputs[i-60:i,0])
    
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
predicted_outcome = model.predict(x_test)
predicted_outcome = msc.inverse_transform(predicted_outcome)


Here comes the most interesting part, Plotting predicted results vs actual results on a matplotlib.pyplot graph helps us visualize and compare the predictions.

In [ ]:
#%% Plot the actual values vs the predicted outcome of stock price on a graph

pt.xlabel('Time')
pt.ylabel('Stock Price')
pt.plot(predicted_outcome, color = 'green', label = 'Predicted Stock Price' )
pt.plot(dataset_test, color = 'blue', label = 'Actual Stock Price' )
pt.legend()
pt.show()
